In [1]:
from dataclasses import dataclass
import numpy as np
from Polynomial import Polynomial, PolynomialTensor

In [2]:
@dataclass
class BfvConfig:
    mat_size: tuple[int, int]
    poly_len: int
    modulus: int
    p: int

    def __post_init__(self):
        self.index_distribution = [(i, j) for i in range(self.mat_size)
                                    for j in range(i, self.mat_size) if j > i]


@dataclass
class BfvSecretKey:
    sk: Polynomial


@dataclass
class BfvPublicKey:
    A: Polynomial
    b: Polynomial


@dataclass
class BfvRlk:
    ra: PolynomialTensor
    rb: PolynomialTensor

In [3]:
from functools import reduce
from operator import add


class BfvEncrypted:

    def __init__(self, config: BfvConfig, rlks: list[BfvRlk], u: Polynomial,
                 v: Polynomial):
        self.config = config
        self.u = u
        self.v = v
        self.rlks = rlks

    def __add__(self, other: "BfvEncrypted") -> "BfvEncrypted":
        assert self.config == other.config
        return BfvEncrypted(self.config,
                            self.rlks,
                            u=(self.u + other.u) % self.config.modulus,
                            v=(self.v + other.v) % self.config.modulus)

    def __mul__(self, other: "BfvEncrypted") -> "BfvEncrypted":
        assert self.config == other.config

        def tp(arr: np.ndarray) -> PolynomialTensor:
            return PolynomialTensor(arr, self.config.modulus)

        d0 = self.v @ other.v
        # print("d0:", d0)
        d1 = (tp([(tp(self.u.poly_mat[i]) @ tp(other.v.poly_mat)).poly_mat[0]
                  for i in range(self.config.mat_size)]) +
              tp([(tp(other.u.poly_mat[i]) @ tp(self.v.poly_mat)).poly_mat[0]
                  for i in range(self.config.mat_size)]))
        # print("d1:", d1)
        d2 = (tp([
            (tp(self.u.poly_mat[i]) @ tp(other.u.poly_mat[i])).poly_mat[0]
            for i in range(self.config.mat_size)
        ]))
        # print("d2:", d2)

        d3 = ([(tp(self.u.poly_mat[i]) @ tp(other.u.poly_mat[j])) +
               (tp(self.u.poly_mat[j]) @ tp(other.u.poly_mat[i]))
               for i, j in self.config.index_distribution])
        # print("d3:", d3)

        assert len(d3) == len(self.rlks) - 1

        ip = 1 / self.config.p
        big_mod = self.config.modulus * self.config.p

        v_relin: PolynomialTensor = ip * reduce(
            add, [
                rlk.rb @ d3i.change_modulus(big_mod)
                for d3i, rlk in zip(d3, self.rlks[1:])
            ],
            d2.change_modulus(big_mod) @ self.rlks[0].rb)

        u_relin: PolynomialTensor = ip * reduce(
            add, [
                rlk.ra @ d3i.change_modulus(big_mod)
                for d3i, rlk in zip(d3, self.rlks[1:])
            ], self.rlks[0].ra @ d2.change_modulus(big_mod))

        return BfvEncrypted(
            self.config,
            self.rlks,
            u=(d1 + u_relin.change_modulus(self.config.modulus)) %
            self.config.modulus,
            v=(d0 + v_relin.change_modulus(self.config.modulus)) %
            self.config.modulus)

In [4]:
class BFV:

    @staticmethod
    def keygen(conf: BfvConfig) -> tuple[BfvSecretKey, BfvPublicKey, BfvRlk]:
        # Key Generation
        s = PolynomialTensor.random_polynomial_matrix(
            conf.poly_len,
            conf.modulus,
            (conf.mat_size, ),
            min_val=-3,
            max_value=3,
        )
        e = PolynomialTensor.random_polynomial_matrix(conf.poly_len,
                                                      conf.modulus,
                                                      (conf.mat_size, ),
                                                      min_val=-3,
                                                      max_value=3)
        A = PolynomialTensor.random_polynomial_matrix(
            conf.poly_len,
            conf.modulus, (conf.mat_size, conf.mat_size),
            min_val=0,
            max_value=conf.modulus)

        b = (-1 * (A @ s + e)) % conf.modulus

        rs = PolynomialTensor(s.poly_mat, conf.p * conf.modulus)

        def _calculate_relin(second_secret: PolynomialTensor) -> BfvRlk:
            re = PolynomialTensor.random_polynomial_matrix(
                conf.poly_len, conf.p * conf.modulus, (conf.mat_size, ), -3, 3)
            ra = PolynomialTensor.random_polynomial_matrix(
                conf.poly_len, conf.p * conf.modulus,
                (conf.mat_size, conf.mat_size))

            rb = (-1 * (ra @ rs + re) + conf.p *
                  (second_secret)) % (conf.p * conf.modulus)
            return BfvRlk(ra=ra, rb=rb)

        relins = [_calculate_relin(rs**2)] + [
            _calculate_relin(
                PolynomialTensor(rs.poly_mat[i], rs.modulus)
                @ PolynomialTensor(rs.poly_mat[j], rs.modulus))
            for i, j in conf.index_distribution
        ]

        return (BfvSecretKey(s), BfvPublicKey(A=A, b=b), relins)

    @staticmethod
    def encrypt(conf: BfvConfig, pk: BfvPublicKey, rlks: list[BfvRlk],
                message: list) -> BfvEncrypted:
        assert isinstance(message, list)
        e1 = PolynomialTensor.random_polynomial_matrix(conf.poly_len,
                                                       conf.modulus, (1, ), -1,
                                                       1)
        e2 = PolynomialTensor.random_polynomial_matrix(conf.poly_len,
                                                       conf.modulus,
                                                       (conf.mat_size, ), -1,
                                                       1)
        r = PolynomialTensor.random_polynomial_matrix(conf.poly_len,
                                                      conf.modulus,
                                                      (conf.mat_size, ), 0, 1)
        dm = Polynomial(
            np.asarray([message]) * (conf.modulus // 2), conf.modulus)
        v = (pk.b.T @ r + e1 + dm) % conf.modulus
        u = (pk.A.T @ r + e2) % conf.modulus

        return BfvEncrypted(conf, rlks, u=u, v=v)

    @staticmethod
    def decrypt(sk: BfvSecretKey, m_enc: BfvEncrypted) -> list:
        return (np.round(
            ((2 / m_enc.config.modulus) *
             ((m_enc.v + m_enc.u @ sk.sk) % m_enc.config.modulus)).poly_mat) %
                2).tolist()[0]

In [23]:
conf = BfvConfig(mat_size=1, poly_len=3, modulus=100, p=1000**5)
sk, pk, rlks = BFV.keygen(conf)

m1 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
m_e1 = BFV.encrypt(conf, pk, rlks, m1.poly_mat[0].tolist())

m2 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
m_e2 = BFV.encrypt(conf, pk, rlks, m2.poly_mat[0].tolist())
# m_e1 = m_e1*m_e2
# m1 = (m1 * m2)%2

print((m1 @ m2)%2)
print(BFV.decrypt(sk, m_e1*m_e2))
assert BFV.decrypt(sk, m_e1*m_e2) == ((m1 @ m2)%2).poly_mat[0].tolist()

0.0 + 1.0·x + 1.0·x²
[0.0, 0.0, 0.0]


AssertionError: 

In [14]:
conf = BfvConfig(1, 3, 1000, 1000**4)

op_count = []
for j in range(50):

    # Single Test Start
    sk, pk, rlks = BFV.keygen(conf)
    m1 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
    m_e1 = BFV.encrypt(conf, pk, rlks, m1.poly_mat[0].tolist())
    for i in range(10000):
        m2 = Polynomial.random_polynomial(conf.poly_len, conf.modulus, 0, 1)
        m_e2 = BFV.encrypt(conf, pk, rlks, m2.poly_mat[0].tolist())
        m_e1 = m_e1*m_e2
        m1 = (m1 @ m2)%2
        # assert BFV.decrypt(sk, m_e1) == m1.poly_mat[0].tolist(), f"{i}: {m1} -- {BFV.decrypt(sk, m_e1)}"
        if BFV.decrypt(sk, m_e1) != m1.poly_mat[0].tolist():
            op_count.append(i)
            break

print("Average Operations:", sum(op_count)/len(op_count))


Average Operations: 0.18


In [ ]:
def tp(arr: np.ndarray)->PolynomialTensor:
    return PolynomialTensor(arr, 1000)

In [ ]:
(
    # v1*v2
    (m_e1.v @ m_e2.v) +
    # v1*u2
    (m_e1.v @ (
        tp(m_e2.u.poly_mat[0]) @ tp(sk.sk.poly_mat[0]) +
        tp(m_e2.u.poly_mat[1]) @ tp(sk.sk.poly_mat[1])
    )) +
    # v2*u1
    (m_e2.v @ (
        (tp(m_e1.u.poly_mat[0]) @ tp(sk.sk.poly_mat[0])) +
        (tp(m_e1.u.poly_mat[1]) @ tp(sk.sk.poly_mat[1]))
    )) +
    # u1*u2 
    (
        (
            # i=0
            (tp(m_e1.u.poly_mat[0]) @ tp(m_e2.u.poly_mat[0]) @ tp(sk.sk.poly_mat[0]) @ tp(sk.sk.poly_mat[0])) +
            (tp(m_e1.u.poly_mat[0]) @ tp(m_e2.u.poly_mat[1]) @ tp(sk.sk.poly_mat[0]) @ tp(sk.sk.poly_mat[1]))
        ) +
        (
            # i=1
            (tp(m_e1.u.poly_mat[1]) @ tp(m_e2.u.poly_mat[0]) @ tp(sk.sk.poly_mat[1]) @ tp(sk.sk.poly_mat[0])) +
            (tp(m_e1.u.poly_mat[1]) @ tp(m_e2.u.poly_mat[1]) @ tp(sk.sk.poly_mat[1]) @ tp(sk.sk.poly_mat[1]))
        )

    )
) % 1000

array([[978., 503., 510.]])

In [ ]:
(
    # d0: just multiply the two scalars
    (m_e1.v @ m_e2.v) +
    # d1: Just multiply each value in the poly vector with the poly scalar leading two a new poly vector. Do this for both and add them
    (
        (
            (
            tp([(tp(m_e2.u.poly_mat[0]) @ tp(m_e1.v.poly_mat)).poly_mat[0],
                (tp(m_e2.u.poly_mat[1]) @ tp(m_e1.v.poly_mat)).poly_mat[0]]) +
            tp([(tp(m_e1.u.poly_mat[0]) @ tp(m_e2.v.poly_mat)).poly_mat[0],
                (tp(m_e1.u.poly_mat[1]) @ tp(m_e2.v.poly_mat)).poly_mat[0]])
            )
        ) @ sk.sk
    ) +
    # d2: Just multiply the parts that are on the same index and create a new vector
    (
        (
            tp([(tp(m_e2.u.poly_mat[0]) @ tp(m_e1.u.poly_mat[0])).poly_mat[0],
                (tp(m_e2.u.poly_mat[1]) @ tp(m_e1.u.poly_mat[1])).poly_mat[0]])
        ) @ sk.sk**2
    ) +
    # d3
    (
        (
            (tp(m_e1.u.poly_mat[0]) @ tp(m_e2.u.poly_mat[1])) +
            (tp(m_e1.u.poly_mat[1]) @ tp(m_e2.u.poly_mat[0]))
        ) @ (tp(sk.sk.poly_mat[1]) @ tp(sk.sk.poly_mat[0]))
    )
) % 1000

array([[978., 503., 510.]])

In [ ]:
print("d0:", (m_e1.v @ m_e2.v))
print("d1:", (
            tp([(tp(m_e2.u.poly_mat[0]) @ tp(m_e1.v.poly_mat)).poly_mat[0],
                (tp(m_e2.u.poly_mat[1]) @ tp(m_e1.v.poly_mat)).poly_mat[0]]) +
            tp([(tp(m_e1.u.poly_mat[0]) @ tp(m_e2.v.poly_mat)).poly_mat[0],
                (tp(m_e1.u.poly_mat[1]) @ tp(m_e2.v.poly_mat)).poly_mat[0]])
            ))
print("d2:", (
            tp([(tp(m_e2.u.poly_mat[0]) @ tp(m_e1.u.poly_mat[0])).poly_mat[0],
                (tp(m_e2.u.poly_mat[1]) @ tp(m_e1.u.poly_mat[1])).poly_mat[0]])
        ))
print("d3:", (
            (tp(m_e1.u.poly_mat[0]) @ tp(m_e2.u.poly_mat[1])) +
            (tp(m_e1.u.poly_mat[1]) @ tp(m_e2.u.poly_mat[0]))
        ))

d0: array([[-205301.,  523011.,  741215.]])
d1: array([[ -598518.,  1253108.,  1423459.],
       [-1015110.,    56841.,  1471795.]])
d2: array([[-204017.,  999422.,  792219.],
       [-737861., -200686.,  413751.]])
d3: array([[-943867.,  364828., 1820443.]])


In [ ]:
mul = (m_e1*m_e2)
(mul.v + sk.sk @ mul.u) % mul.config.modulus

d0: array([[-205301.,  523011.,  741215.]])
d1: array([[ -598518.,  1253108.,  1423459.],
       [-1015110.,    56841.,  1471795.]])
d2: array([[-204017.,  999422.,  792219.],
       [-737861., -200686.,  413751.]])
d3: [array([[-943867.,  364828., 1820443.]])]


array([[740., 708., 993.]])